# How to build an advanced Chatbot with session memory using LangChain
* Advanced Chatbot LLM App.
    * Will be able to have a conversation.
    * Will remember previous interactions: will have memory.
    * Will be able to have different memories for different user sessions.
    * Will be able to remember a limited number of messages: limited memory.


## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages.
    *  LLM Model is based around raw text.
* Chat History: allows Chat Model to remember previous interactions.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 002-advanced-chatbot.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **002-advanced-chatbot**.

## Trick to avoid the nasty deprecation warnings from LangChain

In this exercise we will use the LangChain legacy chain LLMChain. It works well, but LangChain displays a nasty deprecation warning. To avoid it, we will enter the following code:

In [1]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [5]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

In [3]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-4o-mini")

In [4]:
print(chatbot.model_name)

gpt-4o-mini


* Human Message: the user input.

In [5]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favourite color is black."),
]

#### Call the ChatModel (the LLM)

In [6]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="That's great! Black is often associated with elegance, sophistication, and even mystery. It can be a powerful color choice in fashion, design, and art. Do you have a favorite way to incorporate black into your life, like in clothing, decor, or something else?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 13, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjohVORCyBNrfwqSwHdxEoMXb6CkV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--aa277b43-18b0-464f-ad90-137245c9f30e-0', usage_metadata={'input_tokens': 13, 'output_tokens': 54, 'total_tokens': 67, 'input_token_details': {'audio': 0, 'cache_read': 

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Check if the Chatbot remembers your favorite color.

In [7]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I don't have access to personal information, so I can't know your favorite color. However, if you tell me what it is, I'd love to hear about it!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 13, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BjohhW6goE9XHm8e3CfweVQXZtkQB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8487bd14-9e38-4144-8e13-01bff3057887-0', usage_metadata={'input_tokens': 13, 'output_tokens': 33, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

* As you can see, our Chatbot cannot remember our previous interaction.

## Let's add memory to our Chatbot
* We will use the ChatMessageHistory package.
* We will save the Chatbot memory in a python dictionary called chatbotMemory.
* We will define the get_session_history function to create a session_id for each conversation.
* We will use the built-in runnable RunnableWithMesageHistory.

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [10]:
#!pip install langchain_community

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chatbotMemory = {}

# input: session_id, output: chatbotMemory[session_id]
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] = ChatMessageHistory()
    return chatbotMemory[session_id]


chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot, 
    get_session_history
)

#### What is BaseChatMessageHistory and what it does?
BaseChatMessageHistory is what is called an **abstract base class** in Python. [See more info about this here](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html). This means it serves as a template or a foundational **blueprint for other classes**. It outlines a set of methods and structures that any class inheriting from it must implement or adhere to, but it cannot be used to create objects directly.

Here's a simpler breakdown of what it means for `BaseChatMessageHistory` to be an abstract base class:

1. **Blueprint for Other Classes:** It provides a predefined structure that other classes can follow. Think of it like an outline or a checklist for building something; it specifies what needs to be included, but it isn’t the final product.

2. **Cannot Create Instances:** You cannot create an instance of an abstract base class. Trying to create an object directly from `BaseChatMessageHistory` would result in an error because it's meant to be a guide, not something to use directly.

3. **Requires Implementation:** Any class that inherits from this abstract base class needs to implement specific methods outlined in `BaseChatMessageHistory`, such as methods for adding messages, retrieving messages, and clearing messages. The class sets the rules, and the subclasses need to follow these rules by providing the actual operational details.

4. **Purpose in Design:** Using an abstract base class helps ensure consistency and correctness in the implementation of classes that extend it. It's a way to enforce certain functionalities in any subclass, making sure that they all behave as expected without rewriting the same code multiple times.

Overall, the concept of an abstract base class is about setting standards and rules, while leaving the specific details of execution to be defined by subclasses that inherit from it.

#### Let's explain the previous code in simple terms
The previous code manages the chatbot's memory of conversations based on session identifiers. Here’s a breakdown of what the different components do:

1. **chatbotMemory**:
    - `chatbotMemory = {}`: This initializes an empty dictionary where session IDs and their respective chat histories will be stored.

2. **get_session_history Function**:
    - This function, `get_session_history`, takes a `session_id` as an argument and returns the chat history associated with that session.
    - If a chat history for the given `session_id` does not exist in `chatbotMemory`, a new instance of `ChatMessageHistory` is created and assigned to that `session_id` in the dictionary.
    - The function ensures that each session has its own unique chat history, stored and retrieved using the session ID.

3. **chatbot_with_message_history**:
    - `chatbot_with_message_history = RunnableWithMessageHistory(chatbot, get_session_history)`: This line creates an instance of `RunnableWithMessageHistory` using two arguments: `chatbot` and `get_session_history`.
    - The `chatbot` is passed along with the `get_session_history` function. This setup integrates the chatbot with the functionality to handle session-specific chat histories, allowing the chatbot to maintain continuity and context in conversations across different sessions.
    - **Learn more about RunnableWithMessageHistory** [here](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/).

Overall, the code organizes and manages a chatbot's memory, enabling it to handle different sessions with users effectively by remembering previous messages within each session.

#### RunnableWithMessageHistory
**When invoking a new RunnableWithMessageHistory, we specify the corresponding chat history using a configurable parameter**. Let's say we want to create a chat memory for one user session, let's call it session1:

In [9]:
session1 = {"configurable": {"session_id": "001"}}

In [10]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite color is black.")],
    config=session1,
)

responseFromChatbot.content

"That's great! Black is often associated with elegance, sophistication, and versatility. It can convey a sense of mystery and depth as well. Do you have a favorite item or clothing that you love in black?"

In [11]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is black!'

## Let's now change the session_id and see what happens

Now let's create a chat memory for another user session, let's call it session2:

In [12]:
session2 = {"configurable": {"session_id": "002"}}

If the chatbot is using this new memory for session2, it will not be able to remember anything from the previous conversation in the session1:

In [13]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session2,
)

responseFromChatbot.content

"I'm not sure what your favorite color is! If you'd like to share it, I'd be happy to hear."

## Let's go back to session1 and see if the memory is still there

In [14]:
session1 = {"configurable": {"session_id": "001"}}

In [15]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is black!'

As we can see, the chatbot is now able to remember the session1 conversation.

## Our ChatBot has session memory now. Let's check if it remembers the conversation from session2.

In [19]:
session2 = {"configurable": {"session_id": "002"}}

In [16]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mi name is Beto.")],
    config=session2,
)

responseFromChatbot.content

'Nice to meet you, Beto! Do you want to share your favorite color with me?'

In [17]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=session2,
)

responseFromChatbot.content

'Your name is Beto. How can I assist you today?'

In [18]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is black.'

In [19]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=session1,
)

responseFromChatbot.content

'I don’t know your name, but I’d be happy to learn it if you’d like to share!'

## Our chatBot now remembers each of our conversations.

## The importance to manage the Conversation History
* The memory of a chatbot is included in the context window of the LLM so, if left unmanaged, can potentially overflow it.
* **We are now going to learn how to limit the size of the memory of a chatbot**.
* First, let's take a look at what is in the memory of our chatbot:

In [28]:
print(chatbotMemory["002"])

Human: What's my favorite color?
AI: I'm not sure what your favorite color is! If you'd like to share it, I'd be happy to hear.
Human: Mi name is Beto.
AI: Nice to meet you, Beto! Do you want to share your favorite color with me?
Human: What is my name?
AI: Your name is Beto. How can I assist you today?


* Now, **let's define a function to limit the number of messages stored in memory and add it to our chain with .assign**.

In [38]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough


def limited_memory_of_messages(messages, number_of_messages_to_keep=10):
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt 
    | chatbot
)

* The limited_memory_of_messages function allows you to trim the list of stored messages, keeping only a specified number of the latest ones. For example, if you have a long list of messages and you only want to keep the last two, this function will do that for you.
* The lambda function works in conjunction with the `limited_memory_of_messages` function. Here’s a simple breakdown:

    1. **Lambda Function**: The `lambda` keyword is used to create a small anonymous function in Python. The `lambda` function defined here takes one argument, `x`.

    2. **Function Argument**: The argument `x` is expected to be a dictionary that contains a key named `"messages"`. The value associated with this key is a list of messages.

    3. **Function Body**: The body of the `lambda` function calls the `limited_memory_of_messages` function. It passes the list of messages found in `x["messages"]` to this function.

    4. **Default Behavior of limited_memory_of_messages**: Since the `lambda` function does not specify the `number_of_messages_to_keep` parameter when it calls `limited_memory_of_messages`, the latter will default to keeping the last 2 messages from the list (as defined by the earlier function).

In essence, the `lambda` function is a shorthand way to apply the `limited_memory_of_messages` function to the message list contained within a dictionary. It automatically trims the list to the last two messages.

**Let's now create our new chatbot with limited message history**:

In [39]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages",
)

## Let's add 2 more messages to the session1 conversation:

In [31]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite vehicles are Ferraris.")],
    config=session1,
)

responseFromChatbot.content

'Ferraris are fantastic! They’re known for their stunning design, incredible performance, and rich racing heritage. Do you have a favorite Ferrari model?'

In [32]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite city is Vancouver.")],
    config=session1,
)

responseFromChatbot.content

"Vancouver is a beautiful city! It's known for its stunning natural scenery, diverse culture, and vibrant arts scene. Do you have a favorite spot or activity in Vancouver?"

In [47]:
print(chatbot_with_limited_message_history.get_session_history(session_id="001").messages)

[HumanMessage(content='My favorite color is black.', additional_kwargs={}, response_metadata={}), AIMessage(content="That's great! Black is often associated with elegance, sophistication, and versatility. It can convey a sense of mystery and depth as well. Do you have a favorite item or clothing that you love in black?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 13, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjowIeIhU1jH67c9bCZnAxHyBbHkK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--16379b4c-2400-4a72-b56d-ab04e5d303e6-0', usage_metadata={'input_tokens': 13, 'output_tokens': 41, 'total_tokens': 54, 'input_t

## The chatbot memory has now 4 messages. Let's check the Chatbot with limited memory. 
* Remember, this chatbot only remembers the last 2 messages, so if we ask her about the first message she should not remember it.

In [40]:
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my favorite color?")],
    },
    config=session1,
)

responseFromChatbot.content

"I can't determine your favorite color without you telling me. If you share it, I’d be glad to talk about it!"

* The chatbot with limited memory has behaved as we expected.

## Finally, let's compare the previous response with the one provided by the Chatbot with unlimited memory

In [41]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="what is my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is black!'

* As you can see, this chatbot remembers our first message.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 004-invoke-stream-batch.py
* In terminal, make sure you are in the directory of the file and run:
    * python 002-advanced-chatbot.py